# Step 5 - An SDL using Matterhorn Studio  (MacOS)

In this notebook, you will set up a project on Matterhorn Studio and manage data via its API.

## Example: RGB color search

We will use the RGB color search example from SDL4Kids:

In [4]:
import numpy as np
import pandas as pd
import cv2

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize

/opt/homebrew/Caskroom/miniforge/base/envs/SDL4Kids/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd
def calculate_candidate(samples):
    train_X = samples[["R","G","B"]]
    train_Y = samples[["error"]]

    train_X = torch.tensor(train_X.to_numpy(dtype=np.float64))
    train_Y = torch.tensor(-1*train_Y.to_numpy(dtype=np.float64))

    gp = SingleTaskGP(train_X, train_Y, input_transform=Normalize(d=train_X.shape[-1]), outcome_transform=Standardize(m=train_Y.shape[-1]),)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    from botorch.acquisition import UpperConfidenceBound
    UCB = UpperConfidenceBound(gp, beta=0.1)
    
    from botorch.optim import optimize_acqf
    bounds = torch.stack([torch.zeros(3), torch.ones(3)*255])
    candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )
    candidate = candidate[0]
    candidate = {"R": candidate[0], "G": candidate[1], "B": candidate[2]}
    return candidate

# Function to generate random color
import random
def generate_random_color():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return blue, green, red  # OpenCV uses BGR color format

# Function to calculate the error between two colors
def calculate_error(color1, color2):
    return np.sqrt((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2)

In [8]:

# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

# Define the goal color
goal_color = (255,0,255) 

# Main loop
max_iterations = 30  # Set the desired number of iterations

# Let us set up a data table to record the data more structured
samples = pd.DataFrame(columns=['R', 'G', 'B', 'error', 'ID'])

candidate_color = (0,0,0) # Initial color

for iteration in range(max_iterations):
    print(f"---Iteration {iteration}")

    # STEP 1: Synthesis
    # Update the live image with the current color
    print(f"Synthesis:{candidate_color}")
    image[:, :] = tuple(reversed(candidate_color))
    
    # Display the image
    cv2.imshow("Live Image", image)
    
    # STEP 2: Characterisation
    # Measure the color by just plugging it in
    rgb_measurement = candidate_color
    print(f"Characterisation:{rgb_measurement}")
    
    # STEP 3: Planning
    # Calculate the loss between the goal color and the current color
    error = calculate_error(goal_color, rgb_measurement)
    print(f"Error:{error}")

    # Add the sample to the data table
    new_sample = pd.DataFrame({"R":rgb_measurement[0], "G":rgb_measurement[1], "B":rgb_measurement[2], "ID":iteration, "error":error}, index=[iteration])
    samples = pd.concat([samples, new_sample], axis=0, ignore_index=True)
    
    #new_sample = new_sample.copy(deep=True).rename(columns={'R': 'Red', 'G': 'Green', 'B': 'Blue', 'error': 'Error'})
    #client.experiment_update_data(experiment, new_sample)
    

    candidate = calculate_candidate(samples)
    candidate_color = (candidate['R'].item(), candidate['G'].item(), candidate['B'].item())
    print(f"Planning: Candidate:{candidate_color}")
    
    # Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # Plot a subplot for the goal color
    # subplot_x = num_iterations * (width // (max_iterations + 1))
    subplot_x = width // (max_iterations + 1)
    subplot_width = 50
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), tuple(reversed(goal_color)), -1)

    # Add text for the error
    error_text = f"Error: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    error_values = list(samples['error'].to_numpy())    # Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Increment the iteration counter
    cv2.imshow("Live Image", image)
    cv2.waitKey(10)  # Adjust the wait time (in milliseconds) as needed
    
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

---Iteration 0
Synthesis:(0, 0, 0)
Characterisation:(0, 0, 0)
Error:360.62445840513925
Planning: Candidate:(32.5086669921875, 96.50509643554688, 136.71478271484375)
---Iteration 1
Synthesis:(32.5086669921875, 96.50509643554688, 136.71478271484375)
Characterisation:(32.5086669921875, 96.50509643554688, 136.71478271484375)
Error:269.8277590053035
Planning: Candidate:(34.979679107666016, 103.50931549072266, 146.9241485595703)


/opt/homebrew/Caskroom/miniforge/base/envs/SDL4Kids/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


---Iteration 2
Synthesis:(34.979679107666016, 103.50931549072266, 146.9241485595703)
Characterisation:(34.979679107666016, 103.50931549072266, 146.9241485595703)
Error:266.0892888928403
Planning: Candidate:(34.117889404296875, 112.66036224365234, 138.4897918701172)
---Iteration 3
Synthesis:(34.117889404296875, 112.66036224365234, 138.4897918701172)
Characterisation:(34.117889404296875, 112.66036224365234, 138.4897918701172)
Error:273.96330520810994
Planning: Candidate:(35.861873626708984, 91.12440490722656, 156.85064697265625)
---Iteration 4
Synthesis:(35.861873626708984, 91.12440490722656, 156.85064697265625)
Characterisation:(35.861873626708984, 91.12440490722656, 156.85064697265625)
Error:256.82381334249123
Planning: Candidate:(37.46756362915039, 76.78825378417969, 171.5110626220703)
---Iteration 5
Synthesis:(37.46756362915039, 76.78825378417969, 171.5110626220703)
Characterisation:(37.46756362915039, 76.78825378417969, 171.5110626220703)
Error:245.33079598199078
Planning: Candidate

-1

## Creating a Matterhorn Studio Account

1. Start the Tutorial on Matterhorn Studio to create a guest account [here](https://matterhorn.studio/experiments/start_tutorial).
2. Then convert your guest account into a full account [here](https://matterhorn.studio/convert/).
3. Request an API Key [here](https://matterhorn.studio/api_keys/generate) and assign it to the "Token" below:

In [20]:
Token = "Insert Token" # e.g. "67031c6300625373a5a3a0f4576c64592adf1da577a74bf10c8c4ff1315c91e9"

The following code will:
1. Create a project on Matterhorn Studio
2. Create 3 INPUTS: Red, Green, Blue
3. Create 1 OUTPUT: Error
4. Open the page in your browser

In [17]:
# 1. Initialise API client
from MHSapi.MHSapi import MHSapiClient
client = MHSapiClient(token=Token, dev=True)

# 2. Create project
project_data = {'name_text':f'RGB_Example', 'data_table_json':"-"}
project = client.experiments_create(project_data)
print(f"Created:{project}")

# 3. Create INPUTS and OUTPUTS
for p in ['Red', 'Green', 'Blue', 'Error']:
    new_parameter = {"parameter_text": p, 'experiment':project.id}
    new_parameter['lower_bound'] = 255
    new_parameter['upper_bound'] = 255
    new_parameter['reviewed'] = True
    if p == "Error":
        new_parameter["outcome"] = True
    parameter = client.parameters_create(new_parameter)

# 4. Open the project
client.open_experiment(project)

['api_auth_login_create', 'api_auth_logout_create', 'api_auth_logoutall_create', 'api_experiments_create', 'api_experiments_list', 'api_experiments_update', 'api_experiments_destroy', 'api_experiments_retrieve', 'api_experiments_partial_update', 'api_experiments_data_retrieve', 'api_experiments_upload_data_create', 'api_parameters_create', 'api_parameters_list', 'api_parameters_update', 'api_parameters_destroy', 'api_parameters_retrieve', 'api_parameters_partial_update', 'cms_api_main_documents_retrieve', 'cms_api_main_documents_retrieve_2', 'cms_api_main_documents_find_retrieve', 'cms_api_main_images_retrieve', 'cms_api_main_images_retrieve_2', 'cms_api_main_images_find_retrieve', 'cms_api_main_pages_retrieve', 'cms_api_main_pages_retrieve_2', 'cms_api_main_pages_action_create', 'cms_api_main_pages_find_retrieve', 'login_create', 'logout_create', 'logoutall_create']
Created:id=124 url='http://localhost:8000/experiments/124/' name_text='RGB_Example' data_table_json='-'


Let's add a new sample:

In [18]:
new_sample = pd.DataFrame({"Red":0, "Green":0, "Blue":0, "Error":error}, index=[0])
client.experiment_update_data(project, new_sample)

Finally, let's add new samples to Matterhorn Studio after each iteration:

In [19]:

# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

# Define the goal color
goal_color = (255,0,255) 

# Main loop
max_iterations = 30  # Set the desired number of iterations

# Let us set up a data table to record the data more structured
samples = pd.DataFrame(columns=['R', 'G', 'B', 'error', 'ID'])

candidate_color = (0,0,0) # Initial color

for iteration in range(max_iterations):
    print(f"---Iteration {iteration}")

    # STEP 1: Synthesis
    # Update the live image with the current color
    print(f"Synthesis:{candidate_color}")
    image[:, :] = tuple(reversed(candidate_color))
    
    # Display the image
    cv2.imshow("Live Image", image)
    
    # STEP 2: Characterisation
    # Measure the color by just plugging it in
    rgb_measurement = candidate_color
    print(f"Characterisation:{rgb_measurement}")
    
    # STEP 3: Planning
    # Calculate the loss between the goal color and the current color
    error = calculate_error(goal_color, rgb_measurement)
    print(f"Error:{error}")

    # Add the sample to the data table
    new_sample = pd.DataFrame({"R":rgb_measurement[0], "G":rgb_measurement[1], "B":rgb_measurement[2], "ID":iteration, "error":error}, index=[iteration])
    samples = pd.concat([samples, new_sample], axis=0, ignore_index=True)
    
    new_sample = new_sample.copy(deep=True).rename(columns={'R': 'Red', 'G': 'Green', 'B': 'Blue', 'error': 'Error'})
    client.experiment_update_data(project, new_sample)
    

    candidate = calculate_candidate(samples)
    candidate_color = (candidate['R'].item(), candidate['G'].item(), candidate['B'].item())
    print(f"Planning: Candidate:{candidate_color}")
    
    # Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # Plot a subplot for the goal color
    # subplot_x = num_iterations * (width // (max_iterations + 1))
    subplot_x = width // (max_iterations + 1)
    subplot_width = 50
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), tuple(reversed(goal_color)), -1)

    # Add text for the error
    error_text = f"Error: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    error_values = list(samples['error'].to_numpy())    # Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Increment the iteration counter
    cv2.imshow("Live Image", image)
    cv2.waitKey(10)  # Adjust the wait time (in milliseconds) as needed
    
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

---Iteration 0
Synthesis:(0, 0, 0)
Characterisation:(0, 0, 0)
Error:360.62445840513925
Planning: Candidate:(239.69508361816406, 32.588775634765625, 37.791419982910156)
---Iteration 1
Synthesis:(239.69508361816406, 32.588775634765625, 37.791419982910156)
Characterisation:(239.69508361816406, 32.588775634765625, 37.791419982910156)
Error:220.17228707507357


/opt/homebrew/Caskroom/miniforge/base/envs/SDL4Kids/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


Planning: Candidate:(255.0, 35.110652923583984, 40.84807205200195)
---Iteration 2
Synthesis:(255.0, 35.110652923583984, 40.84807205200195)
Characterisation:(255.0, 35.110652923583984, 40.84807205200195)
Error:217.0110738938567
Planning: Candidate:(255.0, 37.24884033203125, 36.740657806396484)
---Iteration 3
Synthesis:(255.0, 37.24884033203125, 36.740657806396484)
Characterisation:(255.0, 37.24884033203125, 36.740657806396484)
Error:221.41503237329138
Planning: Candidate:(234.59422302246094, 36.28727340698242, 42.13291931152344)
---Iteration 4
Synthesis:(234.59422302246094, 36.28727340698242, 42.13291931152344)
Characterisation:(234.59422302246094, 36.28727340698242, 42.13291931152344)
Error:216.89987548683504
Planning: Candidate:(246.66934204101562, 34.151222229003906, 43.812538146972656)
---Iteration 5
Synthesis:(246.66934204101562, 34.151222229003906, 43.812538146972656)
Characterisation:(246.66934204101562, 34.151222229003906, 43.812538146972656)
Error:214.0930869637978
Planning: Ca

-1